# Topic 16: Error Handling and Exceptions

## Overview
Robust error handling is essential for writing reliable Python applications. Learn to anticipate, catch, and handle errors gracefully.

### What You'll Learn:
- Exception types and hierarchy
- try, except, else, finally blocks
- Raising custom exceptions
- Exception best practices
- Logging and debugging exceptions
- Context managers for resource handling

---

## 1. Basic Exception Handling

Understanding try-except blocks:

In [ ]:
# Basic exception handling
print("Basic Exception Handling:")
print("=" * 25)

# Simple try-except
def safe_division(a, b):
    """Safely divide two numbers"""
    try:
        result = a / b
        return result
    except ZeroDivisionError:
        print(f"  Error: Cannot divide {a} by zero!")
        return None

print("Safe division examples:")
print(f"  10 / 2 = {safe_division(10, 2)}")
print(f"  10 / 0 = {safe_division(10, 0)}")

# Multiple exception types
def safe_conversion(value):
    """Safely convert string to integer"""
    try:
        number = int(value)
        reciprocal = 1 / number
        return number, reciprocal
    except ValueError:
        print(f"  Error: '{value}' is not a valid number")
        return None, None
    except ZeroDivisionError:
        print(f"  Error: Cannot calculate reciprocal of zero")
        return int(value), None
    except TypeError:
        print(f"  Error: Expected string or number, got {type(value)}")
        return None, None

test_values = ["5", "0", "abc", None, "10"]
print(f"\nSafe conversion examples:")
for value in test_values:
    number, reciprocal = safe_conversion(value)
    print(f"  '{value}' -> number: {number}, reciprocal: {reciprocal}")

# Catching multiple exceptions
def process_data(data):
    """Process data with multiple potential errors"""
    try:
        # Attempt various operations
        length = len(data)
        first_item = data[0]
        as_number = int(first_item)
        result = 100 / as_number
        return result
    except (TypeError, AttributeError):
        print(f"  Error: Data type issue with {type(data)}")
        return None
    except (IndexError, KeyError):
        print(f"  Error: Data is empty or key missing")
        return None
    except ValueError:
        print(f"  Error: Cannot convert '{first_item}' to number")
        return None
    except ZeroDivisionError:
        print(f"  Error: First item is zero")
        return None

test_data = [
    ["5", "other"],  # Success
    [],               # IndexError
    ["abc"],          # ValueError
    ["0"],            # ZeroDivisionError
    None,             # TypeError
    "string"          # Different behavior
]

print(f"\nData processing examples:")
for data in test_data:
    result = process_data(data)
    print(f"  {data} -> {result}")

## 2. Exception Hierarchy and Specific Handling

Understanding Python's exception hierarchy:

In [ ]:
# Exception hierarchy and specific handling
print("Exception Hierarchy:")
print("=" * 19)

# Common built-in exceptions
exceptions_demo = [
    (lambda: 1/0, "ZeroDivisionError"),
    (lambda: int("abc"), "ValueError"),
    (lambda: [1,2,3][10], "IndexError"),
    (lambda: {"a": 1}["b"], "KeyError"),
    (lambda: len(None), "TypeError"),
    (lambda: open("nonexistent_file.txt"), "FileNotFoundError"),
    (lambda: import_nonexistent_module, "NameError"),
]

print("Common exceptions:")
for func, expected_error in exceptions_demo:
    try:
        func()
    except Exception as e:
        print(f"  {expected_error}: {e}")

# Exception hierarchy matters for catching
def demonstrate_hierarchy():
    """Show how exception hierarchy affects catching"""
    
    # Specific to general (good practice)
    def good_handling(value):
        try:
            return int(value) / 0
        except ZeroDivisionError:
            return "Caught ZeroDivisionError"
        except ValueError:
            return "Caught ValueError"
        except Exception:
            return "Caught general exception"
    
    # General to specific (problematic)
    def bad_handling(value):
        try:
            return int(value) / 0
        except Exception:  # This catches everything first!
            return "Caught general exception"
        except ZeroDivisionError:  # This will never be reached
            return "Caught ZeroDivisionError"
        except ValueError:  # This will never be reached
            return "Caught ValueError"
    
    test_cases = ["0", "abc"]
    
    print(f"\nException handling order:")
    for value in test_cases:
        good_result = good_handling(value)
        bad_result = bad_handling(value)
        print(f"  '{value}' - Good: {good_result}")
        print(f"  '{value}' - Bad:  {bad_result}")

demonstrate_hierarchy()

# Accessing exception information
def detailed_error_info():
    """Demonstrate accessing exception details"""
    try:
        result = int("not_a_number")
    except ValueError as e:
        print(f"\nDetailed error information:")
        print(f"  Exception type: {type(e).__name__}")
        print(f"  Exception message: {e}")
        print(f"  Exception args: {e.args}")
        
        # Get traceback information
        import traceback
        print(f"  Traceback:")
        traceback.print_exc()

detailed_error_info()

# Exception inheritance
print(f"\nException inheritance example:")
try:
    # This will raise FileNotFoundError
    with open("nonexistent.txt", "r") as f:
        content = f.read()
except OSError as e:  # FileNotFoundError inherits from OSError
    print(f"  Caught as OSError: {e}")
except Exception as e:
    print(f"  Caught as general Exception: {e}")

# Check inheritance
print(f"\nInheritance relationships:")
print(f"  FileNotFoundError is subclass of OSError: {issubclass(FileNotFoundError, OSError)}")
print(f"  OSError is subclass of Exception: {issubclass(OSError, Exception)}")
print(f"  ValueError is subclass of Exception: {issubclass(ValueError, Exception)}")

# Exception MRO (Method Resolution Order)
print(f"\nFileNotFoundError MRO:")
for i, cls in enumerate(FileNotFoundError.__mro__):
    print(f"  {i}: {cls.__name__}")

## 3. Try-Except-Else-Finally Blocks

Complete exception handling structure:

In [ ]:
# Try-except-else-finally blocks
print("Complete Exception Handling Structure:")
print("=" * 38)

# Demonstrating all blocks
def complete_exception_demo(filename, content=None):
    """Demonstrate try-except-else-finally structure"""
    file_handle = None
    
    try:
        print(f"  Trying to open file: {filename}")
        file_handle = open(filename, 'w' if content else 'r')
        
        if content:
            file_handle.write(content)
            print(f"  Successfully wrote to {filename}")
        else:
            data = file_handle.read()
            print(f"  Successfully read from {filename}: {data[:50]}...")
            
    except FileNotFoundError:
        print(f"  Error: File {filename} not found")
        return False
        
    except PermissionError:
        print(f"  Error: Permission denied for {filename}")
        return False
        
    except Exception as e:
        print(f"  Unexpected error: {e}")
        return False
        
    else:
        # Executes only if no exception occurred
        print(f"  Success: File operation completed successfully")
        return True
        
    finally:
        # Always executes, even if return statements above
        if file_handle and not file_handle.closed:
            file_handle.close()
            print(f"  Cleanup: File handle closed")
        print(f"  Finally block executed")

print("Example 1 - Creating and reading a file:")
# Create a test file
success1 = complete_exception_demo("test_file.txt", "Hello, World!\nThis is a test file.")
print(f"Result: {success1}")

print("\nExample 2 - Reading the created file:")
success2 = complete_exception_demo("test_file.txt")
print(f"Result: {success2}")

print("\nExample 3 - Trying to read non-existent file:")
success3 = complete_exception_demo("nonexistent.txt")
print(f"Result: {success3}")

# Else block behavior
print(f"\nElse block demonstration:")

def division_with_else(a, b):
    """Demonstrate else block in exception handling"""
    try:
        result = a / b
    except ZeroDivisionError:
        print(f"    Cannot divide by zero")
        return None
    else:
        # This runs only if no exception occurred
        print(f"    Division successful: {a} / {b} = {result}")
        return result
    finally:
        print(f"    Division attempt completed")

test_cases = [(10, 2), (10, 0), (15, 3)]
for a, b in test_cases:
    print(f"  Dividing {a} by {b}:")
    result = division_with_else(a, b)
    print(f"  Final result: {result}\n")

# Exception in finally block (be careful!)
print(f"Exception in finally block (problematic):")

def problematic_finally():
    """Show what happens when finally block has exception"""
    try:
        print("  In try block")
        raise ValueError("Original exception")
    except ValueError:
        print("  In except block")
        raise RuntimeError("Exception in except")
    finally:
        print("  In finally block")
        # raise TypeError("Exception in finally")  # This would mask the RuntimeError!
        print("  Finally completed safely")

try:
    problematic_finally()
except Exception as e:
    print(f"  Caught final exception: {type(e).__name__}: {e}")

# Nested try-except blocks
print(f"\nNested try-except blocks:")

def nested_exception_handling():
    """Demonstrate nested exception handling"""
    try:
        print("  Outer try block")
        try:
            print("    Inner try block")
            value = int("not_a_number")
        except ValueError:
            print("    Inner except: Caught ValueError")
            # Re-raise or create new exception
            raise RuntimeError("Converted from ValueError")
        except Exception as e:
            print(f"    Inner except: Caught {type(e).__name__}")
            raise
    except RuntimeError as e:
        print(f"  Outer except: Caught RuntimeError: {e}")
    except Exception as e:
        print(f"  Outer except: Caught {type(e).__name__}: {e}")
    finally:
        print("  Outer finally block")

nested_exception_handling()

## 4. Raising and Creating Custom Exceptions

Creating your own exception types:

In [ ]:
# Raising and creating custom exceptions
print("Custom Exceptions:")
print("=" * 17)

# Raising built-in exceptions
def validate_age(age):
    """Validate age with custom error messages"""
    if not isinstance(age, (int, float)):
        raise TypeError(f"Age must be a number, got {type(age).__name__}")
    
    if age < 0:
        raise ValueError("Age cannot be negative")
    
    if age > 150:
        raise ValueError("Age cannot be greater than 150")
    
    return True

test_ages = [25, -5, 200, "twenty-five", 150.5]
print("Age validation:")
for age in test_ages:
    try:
        validate_age(age)
        print(f"  {age}: Valid")
    except (TypeError, ValueError) as e:
        print(f"  {age}: Invalid - {e}")

# Custom exception classes
class CustomError(Exception):
    """Base class for custom exceptions"""
    pass

class ValidationError(CustomError):
    """Raised when validation fails"""
    def __init__(self, message, field=None, value=None):
        super().__init__(message)
        self.field = field
        self.value = value
        self.message = message
    
    def __str__(self):
        if self.field:
            return f"Validation error in '{self.field}': {self.message} (got: {self.value})"
        return self.message

class BusinessLogicError(CustomError):
    """Raised when business logic is violated"""
    def __init__(self, message, error_code=None):
        super().__init__(message)
        self.error_code = error_code
        self.message = message
    
    def __str__(self):
        if self.error_code:
            return f"[{self.error_code}] {self.message}"
        return self.message

# Using custom exceptions
class User:
    """User class with validation"""
    
    def __init__(self, username, email, age):
        self.username = self._validate_username(username)
        self.email = self._validate_email(email)
        self.age = self._validate_age(age)
    
    def _validate_username(self, username):
        if not isinstance(username, str):
            raise ValidationError("Username must be a string", "username", username)
        
        if len(username) < 3:
            raise ValidationError("Username too short (min 3 chars)", "username", username)
        
        if not username.isalnum():
            raise ValidationError("Username must be alphanumeric", "username", username)
        
        return username
    
    def _validate_email(self, email):
        if not isinstance(email, str):
            raise ValidationError("Email must be a string", "email", email)
        
        if '@' not in email:
            raise ValidationError("Email must contain @", "email", email)
        
        return email
    
    def _validate_age(self, age):
        if not isinstance(age, (int, float)):
            raise ValidationError("Age must be a number", "age", age)
        
        if age < 0:
            raise ValidationError("Age cannot be negative", "age", age)
        
        if age > 120:
            raise ValidationError("Age seems unrealistic", "age", age)
        
        return age
    
    def change_username(self, new_username):
        if new_username == self.username:
            raise BusinessLogicError("New username same as current", "SAME_USERNAME")
        
        self.username = self._validate_username(new_username)
    
    def __str__(self):
        return f"User({self.username}, {self.email}, {self.age})"

# Test custom exceptions
test_users = [
    ("alice123", "alice@example.com", 25),     # Valid
    ("ab", "valid@email.com", 30),             # Username too short
    ("validuser", "invalid_email", 25),        # Invalid email
    ("gooduser", "good@email.com", -5),        # Negative age
    (123, "email@test.com", 25),               # Non-string username
]

print(f"\nUser creation tests:")
valid_users = []
for username, email, age in test_users:
    try:
        user = User(username, email, age)
        valid_users.append(user)
        print(f"  ✓ Created: {user}")
    except ValidationError as e:
        print(f"  ✗ Validation failed: {e}")
    except Exception as e:
        print(f"  ✗ Unexpected error: {e}")

# Test business logic exceptions
if valid_users:
    print(f"\nBusiness logic tests:")
    user = valid_users[0]
    try:
        user.change_username(user.username)  # Same username
    except BusinessLogicError as e:
        print(f"  ✗ Business logic error: {e}")
    
    try:
        user.change_username("newname123")
        print(f"  ✓ Username changed successfully")
    except ValidationError as e:
        print(f"  ✗ Validation error: {e}")

# Exception chaining (Python 3+)
print(f"\nException chaining:")

def process_file(filename):
    """Demonstrate exception chaining"""
    try:
        with open(filename, 'r') as f:
            data = f.read()
            result = int(data)
            return result
    except FileNotFoundError as e:
        raise BusinessLogicError("Configuration file missing") from e
    except ValueError as e:
        raise BusinessLogicError("Invalid configuration format") from e

try:
    result = process_file("nonexistent_config.txt")
except BusinessLogicError as e:
    print(f"  Business error: {e}")
    print(f"  Caused by: {e.__cause__}")
    print(f"  Exception chain: {type(e.__cause__).__name__} -> {type(e).__name__}")

## 5. Exception Best Practices

Writing robust exception handling code:

In [ ]:
# Exception handling best practices
print("Exception Handling Best Practices:")
print("=" * 35)

# Practice 1: Be specific with exception types
print("1. Be specific with exceptions:")

# Bad: Catching all exceptions
def bad_file_reader(filename):
    """Bad example - too broad exception handling"""
    try:
        with open(filename, 'r') as f:
            return f.read()
    except Exception:  # Too broad!
        print("Something went wrong")
        return None

# Good: Catching specific exceptions
def good_file_reader(filename):
    """Good example - specific exception handling"""
    try:
        with open(filename, 'r') as f:
            return f.read()
    except FileNotFoundError:
        print(f"File {filename} not found")
        return None
    except PermissionError:
        print(f"Permission denied reading {filename}")
        return None
    except UnicodeDecodeError:
        print(f"File {filename} has encoding issues")
        return None
    except Exception as e:
        print(f"Unexpected error reading {filename}: {e}")
        raise  # Re-raise unexpected exceptions

print("  Good approach handles specific cases and re-raises unexpected errors")

# Practice 2: Don't ignore exceptions
print("\n2. Don't ignore exceptions:")

# Bad: Silent failure
def bad_convert_to_int(value):
    """Bad example - silent failure"""
    try:
        return int(value)
    except:
        pass  # Silent failure - BAD!

# Good: Handle or propagate
def good_convert_to_int(value, default=None):
    """Good example - explicit handling"""
    try:
        return int(value)
    except ValueError:
        if default is not None:
            return default
        raise  # Re-raise if no default provided
    except TypeError:
        raise TypeError(f"Cannot convert {type(value).__name__} to int")

test_values = ["123", "abc", None]
for value in test_values:
    try:
        result = good_convert_to_int(value, default=0)
        print(f"  '{value}' -> {result}")
    except Exception as e:
        print(f"  '{value}' -> Error: {e}")

# Practice 3: Use finally for cleanup
print("\n3. Use finally for cleanup:")

class Resource:
    """Simulated resource that needs cleanup"""
    def __init__(self, name):
        self.name = name
        self.is_open = False
    
    def open(self):
        self.is_open = True
        print(f"    Resource {self.name} opened")
    
    def close(self):
        if self.is_open:
            self.is_open = False
            print(f"    Resource {self.name} closed")
    
    def process(self):
        if not self.is_open:
            raise RuntimeError(f"Resource {self.name} not open")
        print(f"    Processing {self.name}")
        # Simulate potential error
        if self.name == "problematic":
            raise ValueError("Processing failed")

# Good resource management
def process_resource(resource_name):
    """Process resource with proper cleanup"""
    resource = Resource(resource_name)
    try:
        resource.open()
        resource.process()
        return True
    except Exception as e:
        print(f"    Error processing {resource_name}: {e}")
        return False
    finally:
        resource.close()  # Always cleanup

resource_names = ["normal", "problematic"]
for name in resource_names:
    print(f"  Processing {name}:")
    success = process_resource(name)
    print(f"  Success: {success}\n")

# Practice 4: Log exceptions properly
print("4. Log exceptions properly:")

import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

def divide_with_logging(a, b):
    """Division with proper logging"""
    try:
        result = a / b
        logger.info(f"Division successful: {a} / {b} = {result}")
        return result
    except ZeroDivisionError:
        logger.error(f"Division by zero: {a} / {b}")
        raise
    except Exception as e:
        logger.exception(f"Unexpected error in division: {a} / {b}")
        raise

print("  Check logs for division operations:")
for a, b in [(10, 2), (10, 0)]:
    try:
        result = divide_with_logging(a, b)
    except:
        pass  # Error already logged

# Practice 5: Use context managers for resource management
print("\n5. Use context managers:")

class ManagedResource:
    """Resource with context manager support"""
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        print(f"    Acquiring resource: {self.name}")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"    Releasing resource: {self.name}")
        if exc_type:
            print(f"    Exception occurred: {exc_type.__name__}")
        return False  # Don't suppress exceptions
    
    def work(self):
        print(f"    Working with {self.name}")
        if self.name == "failing":
            raise RuntimeError("Work failed")

# Using context managers
for resource_name in ["working", "failing"]:
    print(f"  Using resource '{resource_name}':")
    try:
        with ManagedResource(resource_name) as resource:
            resource.work()
        print("    Success\n")
    except Exception as e:
        print(f"    Error: {e}\n")

print("Best practices summary:")
print("  ✓ Catch specific exceptions, not all exceptions")
print("  ✓ Don't ignore exceptions silently")
print("  ✓ Use finally for cleanup or context managers")
print("  ✓ Log exceptions with context")
print("  ✓ Re-raise unexpected exceptions")
print("  ✓ Provide meaningful error messages")

## Summary

In this notebook, you learned about:

✅ **Basic Exception Handling**: try-except blocks and multiple exception types  
✅ **Exception Hierarchy**: Understanding inheritance and specific vs general catching  
✅ **Complete Structure**: try-except-else-finally blocks and their execution order  
✅ **Custom Exceptions**: Creating meaningful application-specific error types  
✅ **Best Practices**: Writing robust, maintainable error handling code  
✅ **Resource Management**: Using finally blocks and context managers  

### Key Takeaways:
1. Catch specific exceptions, not general ones
2. Never ignore exceptions silently
3. Use finally for cleanup operations
4. Create custom exceptions for better error communication
5. Log exceptions with proper context
6. Use context managers for resource management

### Next Topic: 17_modules_packages.ipynb
Learn about organizing code into modules and packages.